In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    
    features = {
        'filename': os.path.basename(file_path),
        'length': len(y) / sr  # Duration in seconds
    }
    
    # Spectral features
    def add_feature(feature_func, name, **kwargs):
        vals = feature_func(y=y, sr=sr, **kwargs)
        features[f'{name}_mean'] = np.mean(vals)
        features[f'{name}_var'] = np.var(vals)
    
    add_feature(librosa.feature.chroma_stft, 'chroma_stft')
    add_feature(librosa.feature.rms, 'rms')
    add_feature(librosa.feature.spectral_centroid, 'spectral_centroid')
    add_feature(librosa.feature.spectral_bandwidth, 'spectral_bandwidth')
    add_feature(librosa.feature.spectral_rolloff, 'rolloff')
    add_feature(librosa.feature.zero_crossing_rate, 'zero_crossing_rate')
    
    # Harmony/Percussive
    harmony, perceptr = librosa.effects.hpss(y)
    features.update({
        'harmony_mean': np.mean(harmony),
        'harmony_var': np.var(harmony),
        'perceptr_mean': np.mean(perceptr),
        'perceptr_var': np.var(perceptr)
    })
    
    # Tempo
    features['tempo'] = librosa.beat.beat_track(y=y, sr=sr)[0]
    
    # MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    for i in range(20):
        features[f'mfcc{i+1}_mean'] = np.mean(mfccs[i])
        features[f'mfcc{i+1}_var'] = np.var(mfccs[i])
    
    return features

def build_dataset(dataset_path, genres):
    all_features = []
    
    for genre in genres:
        genre_path = os.path.join(dataset_path, genre)
        if not os.path.isdir(genre_path):
            continue
            
        for file in os.listdir(genre_path):
            if file.endswith(".wav"):
                try:
                    features = extract_features(os.path.join(genre_path, file))
                    features['label'] = genre
                    all_features.append(features)
                except Exception as e:
                    print(f"Error processing {file}: {str(e)}")
                    continue
                    
    return pd.DataFrame(all_features)

# Usage
DATASET_PATH = "Data/genres_original"
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']

# Extract features and save
df = build_dataset(DATASET_PATH, genres)
df.to_csv('audio_features.csv', index=False)

# Load and prepare for modeling
data = pd.read_csv('audio_features.csv')

# Ensure numeric columns are properly typed
numeric_cols = data.columns.difference(['filename', 'label'])
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label'])

# Prepare X and y
X = data.drop(columns=['filename', 'label', 'label_encoded'])
y = data['label_encoded']

In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm

def extract_clip_features(y, sr, clip_num, original_filename):
    """Extract features for a single 3-second audio clip"""
    features = {
        'filename': f"{os.path.splitext(original_filename)[0]}.{clip_num}.wav",
        'length': len(y)  # Number of samples
    }
    
    # Helper function to add mean/var features
    def add_feature(feature_func, name, **kwargs):
        vals = feature_func(y=y, **kwargs) if name != 'rms' else feature_func(y=y)  # Special handling for RMS
        features[f'{name}_mean'] = np.mean(vals)
        features[f'{name}_var'] = np.var(vals)
    
    # Spectral features (remove sr from RMS call)
    add_feature(librosa.feature.chroma_stft, 'chroma_stft', sr=sr)
    add_feature(librosa.feature.rms, 'rms')  # Removed sr parameter
    add_feature(librosa.feature.spectral_centroid, 'spectral_centroid', sr=sr)
    add_feature(librosa.feature.spectral_bandwidth, 'spectral_bandwidth', sr=sr)
    add_feature(librosa.feature.spectral_rolloff, 'rolloff', sr=sr)
    add_feature(librosa.feature.zero_crossing_rate, 'zero_crossing_rate')
    
    # Harmony/Percussive
    harmony, perceptr = librosa.effects.hpss(y)
    features.update({
        'harmony_mean': np.mean(harmony),
        'harmony_var': np.var(harmony),
        'perceptr_mean': np.mean(perceptr),
        'perceptr_var': np.var(perceptr)
    })
    
    # Tempo
    tempo = librosa.beat.beat_track(y=y, sr=sr)[0]
    features['tempo'] = float(tempo)
    
    # MFCCs (20 coefficients)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    for i in range(20):
        features[f'mfcc{i+1}_mean'] = np.mean(mfccs[i])
        features[f'mfcc{i+1}_var'] = np.var(mfccs[i])
    
    return features

def process_audio_file(file_path, clip_duration=3.0):
    """Split audio file into clips and extract features for each"""
    try:
        y, sr = librosa.load(file_path, sr=None)
        duration = librosa.get_duration(y=y, sr=sr)
        clip_samples = int(clip_duration * sr)
        total_clips = min(10, int(duration // clip_duration))
        
        all_features = []
        for i in range(total_clips):
            start = i * clip_samples
            end = start + clip_samples
            clip = y[start:end]
            
            if len(clip) < clip_samples:
                clip = np.pad(clip, (0, max(0, clip_samples - len(clip))))
            
            features = extract_clip_features(clip, sr, i, os.path.basename(file_path))
            all_features.append(features)
        
        return all_features
    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {str(e)}")
        return []

def build_dataset(dataset_path, genres):
    """Process all audio files in the dataset"""
    all_features = []
    
    for genre in tqdm(genres, desc="Processing genres"):
        print(genre)
        genre_path = os.path.join(dataset_path, genre)
        if not os.path.isdir(genre_path):
            continue
            
        for file in tqdm(os.listdir(genre_path), desc=f"Processing {genre}", leave=False):
            if file.endswith(".wav"):
                file_path = os.path.join(genre_path, file)
                clip_features = process_audio_file(file_path)
                for feat in clip_features:
                    feat['label'] = genre
                all_features.extend(clip_features)
                    
    return pd.DataFrame(all_features) if all_features else pd.DataFrame()

# Configuration
DATASET_PATH = "Data/genres_original"
genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 
          'metal', 'pop', 'reggae', 'rock']
OUTPUT_CSV = 'audio_features_clips.csv'

# Process dataset
print("Starting feature extraction...")
df = build_dataset(DATASET_PATH, genres)

if not df.empty:
    # Save to CSV
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"Feature extraction complete. Saved {len(df)} clips to {OUTPUT_CSV}")
    
    # Verify the output
    print("\nSample features:")
    print(df.iloc[0])
else:
    print("Error: No features were extracted. Please check your input files and paths.")

Starting feature extraction...


Processing genres:   0%|          | 0/10 [00:00<?, ?it/s]

blues


Data/genres_original\blues\blues.00000.wav


Data/genres_original\blues\blues.00001.wav


...

Data/genres_original\rock\rock.00097.wav


Data/genres_original\rock\rock.00098.wav


Data/genres_original\rock\rock.00099.wav


Processing genres: 100%|██████████| 10/10 [21:51<00:00, 131.18s/it]


Feature extraction complete. Saved 9991 clips to audio_features_clips.csv

Sample features:
filename                   blues.00000.0.wav
length                                 66150
chroma_stft_mean                    0.335555
chroma_stft_var                     0.090997
rms_mean                            0.130189
rms_var                             0.003559
spectral_centroid_mean           1773.358004
spectral_centroid_var          169450.829707
spectral_bandwidth_mean          1972.334258
spectral_bandwidth_var         117272.640573
rolloff_mean                     3714.063439
rolloff_var                   1083179.450351
zero_crossing_rate_mean             0.081851
zero_crossing_rate_var              0.000558
harmony_mean                       -0.000047
harmony_var                         0.008388
perceptr_mean                       -0.00006
perceptr_var                        0.005511
tempo                         [129.19921875]
mfcc1_mean                       -119.018044
mfcc1_va

In [6]:
import pandas as pd

# Read CSV and clean tempo column
df = pd.read_csv('audio_features_clips.csv')
df['tempo'] = df['tempo'].str.strip('[]').astype(float)
# Save with numeric formatting
df.to_csv('audio_features_clips.csv', float_format='%.6f', index=False)